- 문제: https://school.programmers.co.kr/learn/courses/30/lessons/72412

## 풀이 1: 정확성 테스트 O, 효율성 테스트 X

In [9]:
%%timeit
from bisect import bisect_left

def exclude_and(value):
    return value != 'and'

def convert_to_int_if_digit(value):
    return int(value) if value.isdigit() else value

def solution(infos, query):
    answer = []
    
    # 지원자 정보 string을 list자료형으로 변환
    infos = [list(map(convert_to_int_if_digit, info.split())) for info in infos]
    
    infos.sort(key = lambda x:x[-1]) # 지원자 점수순으로 정렬
        
    for q in query:
        # query문을 list 자료형으로 변환
        processed_query = list(filter(exclude_and, q.split()))
        processed_query = list(map(convert_to_int_if_digit, processed_query))
        score_cut = processed_query[-1] # 넘어야 하는 점수
        
        target = bisect_left([info[-1] for info in infos], score_cut)
        
        result = 0
        for i in range(target, len(infos)):
            for k in range(0, 4):
                if processed_query[k] == '-':
                    continue
                
                if processed_query[k] != infos[i][k]:
                    break
            else:
                result += 1
        
        answer.append(result)
        
    return answer

solution(["java backend junior pizza 150", "python frontend senior chicken 210", "python frontend senior chicken 150", "cpp backend senior pizza 260", "java backend junior chicken 80", "python backend senior chicken 50"], ["java and backend and junior and pizza 100", "python and frontend and senior and chicken 200", "cpp and - and senior and pizza 250", "- and backend and senior and - 150", "- and - and - and chicken 100", "- and - and - and - 150"])

37.2 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 풀이 2: 이진 탐색 -> 정확성 테스트 O, 효율성 테스트 O

우선, 매 문의 조건마다 INFO 배열에서 조건에 해당하는 지원자들을 찾으면서 X점 이상 받은 사람은 몇 명인지 구한다면 정확성 테스트를 통과할 수 있습니다.
그러나 효율성 테스트의 경우에는 위와 같은 방식으로 매번 지원자들을 찾는다면 통과할 수 없습니다. 문제 해결을 위해서, 지원자들을 그룹별로 적절하게 미리 분류해두면 매 문의 조건마다 지원자들을 INFO 배열에서 찾지 않아도 됩니다.

예를 들어, “java backend junior pizza 150” 지원자의 경우 16가지 그룹에 속하게 됩니다.

In [10]:
%%timeit

from collections import defaultdict
from bisect import bisect_left

def solution(infos, queries):
    answer = []
    
    # 정보를 기반으로 가능한 모든 조합을 생성하고 점수를 저장
    info_dict = defaultdict(list)
    for info in infos:
        info_parts = info.split()
        score = int(info_parts[-1])
        info_parts = info_parts[:-1]

        # 각 부분에서 '-'를 포함한 모든 조합 생성
        for lang in [info_parts[0], '-']:
            for part in [info_parts[1], '-']:
                for career in [info_parts[2], '-']:
                    for food in [info_parts[3], '-']:
                        info_key = ''.join([lang, part, career, food])
                        info_dict[info_key].append(score)
    
    # 각 조합별 점수를 정렬하여 이진 탐색 준비
    for scores in info_dict.values():
        scores.sort()
        
    for query in queries:
        # 'and'를 제거하여 query_key 생성
        query_parts = query.replace(' and', '').split()
        query_score = int(query_parts[-1])
        query_key = ''.join(query_parts[:-1])
        
        # 점수 기준 이진 탐색으로 해당 점수 이상의 지원자 수 계산
        scores = info_dict[query_key]
        pass_count = len(scores) - bisect_left(scores, query_score)
        
        answer.append(pass_count)
        
    return answer

solution(["java backend junior pizza 150", "python frontend senior chicken 210", "python frontend senior chicken 150", "cpp backend senior pizza 260", "java backend junior chicken 80", "python backend senior chicken 50"], ["java and backend and junior and pizza 100", "python and frontend and senior and chicken 200", "cpp and - and senior and pizza 250", "- and backend and senior and - 150", "- and - and - and chicken 100", "- and - and - and - 150"])

61 µs ± 1.85 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
